In [12]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

kwkwdf = pd.read_excel("키워드 리스트.xlsx")
keywordlist = list(kwkwdf.키워드.unique())
print(len(keywordlist))
keywordlst = []
for kk in keywordlist:
    keywordlst.append(kk.replace(' ',''))
print(len(keywordlst))
keywordlst

54
54


['혈압정상수치',
 '고지혈증',
 '혈압낮추는법',
 '고혈압',
 '고혈압증상',
 '혈당정상수치',
 '고혈압낮추는방법',
 '고지혈증좋은음식',
 '혈압',
 '고혈압에좋은음식',
 '고지혈증증상',
 '고지혈증약',
 '혈압높을때증상',
 '혈압약',
 '고혈압원인',
 '고혈압약',
 '혈압낮추는음식',
 '심혈관질환',
 '혈당낮추는음식',
 '고지혈증약부작용',
 '혈당낮추는법',
 '혈압에좋은음식',
 '심혈관질환증상',
 '고지혈증원인',
 '뇌출혈증상',
 '고혈압식단',
 '혈압에좋은영양제',
 '고지혈증에좋은음식',
 '고혈압에좋은영양제',
 '고혈압관리',
 '혈압영양제',
 '뇌경색',
 '뇌경색전조증상',
 '뇌경색초기증상',
 '뇌경색치료',
 '뇌경색증상',
 '뇌경색후유증',
 '뇌경색원인',
 '뇌경색좋은음식',
 '뇌출혈초기증상',
 '뇌출혈',
 '뇌출혈전조증상',
 '뇌출혈원인',
 '뇌출혈증상',
 '뇌출혈후유증',
 '뇌졸중초기증상',
 '뇌졸중',
 '뇌졸중전조증상',
 '뇌졸증',
 '뇌졸증증상',
 '뇌졸중증상',
 '뇌졸증전조증상',
 '뇌졸중원인',
 '뇌졸증증세']

In [13]:
# keywordlst = []
resultlst = []
for kw in keywordlst:
    dic = {}
    dic['키워드'] = kw
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query={}'.format(kw)
    res = requests.get(url)
    page = BeautifulSoup(res.text,'html.parser')
    ll = page.find_all("h2")
    pagelst = []
    for ilil in ll:
        kk = ilil.text
        kk = kk.replace("이 정보가 표시된 이유",'')
        kk = kk.replace("네이버가 운영하는 쇼핑 서비스입니다.",'')
        kk = kk.replace('정보확인 레이어 닫기','')
        kk = kk.replace('네이버 인플루언서로 활동하는 창작자의 콘텐츠를 검색 결과에서 제공합니다.','')
        kk = kk.replace("자세히 보기",'')
        kk = kk.replace(" ",'')

        pagelst.append(kk)
        
        
    ll = page.find_all('section')
    illst = []
    for ilil in ll:
        if ilil.find('h2') != None: 
            kk = ilil.find('h2').text
            kk = kk.replace("이 정보가 표시된 이유",'')
            kk = kk.replace("네이버가 운영하는 쇼핑 서비스입니다.",'')
            kk = kk.replace('정보확인 레이어 닫기','')
            kk = kk.replace('네이버 인플루언서로 활동하는 창작자의 콘텐츠를 검색 결과에서 제공합니다.','')
            kk = kk.replace("자세히 보기",'')
            kk = kk.replace(" ",'')
            name = kk
            illst.append(name)
        else:
            try:
                name = ilil.find('strong').text
                print(name)
                illst.append(name)
            except:
                if ilil.find('div',{'class':'snippet_dsc api_ani_send'}) != None:
                    name = '지식스니펫'
                    print(ll.index(ilil), name)
                    illst.append(name)
                else:
                    print('잉')
                    pass


    pagelst2 = pagelst
    i = 1
    for dd in illst:
        
        if dd in pagelst2:
            i = pagelst2.index(dd) + 1
        else:
    #         print(dd)
    #         print(i)
            pagelst2.insert(i,dd)
            i += 1

    
    if '알림' in pagelst2:
        pagelst2.remove('알림')
    
    if '인플루언서' in pagelst2 or '인플루언서 참여 콘텐츠' in pagelst2:
        dic['인플루언서 노출 여부'] = 'o'
        try:
            dic['view(인플루언서)'] = pagelst2.index('인플루언서') + 1
        except:
            dic['view(인플루언서)'] = pagelst2.index('인플루언서 참여 콘텐츠') + 1
    else:
        dic['인플루언서 노출 여부'] = 'x'
        try:
            dic['view(인플루언서)'] = pagelst2.index('VIEW') + 1
            viewsection = page.find('section',{'class':'sc_new sp_nreview _prs_rvw _au_view_collection'})
            viewdata = viewsection.find_all('div',{'class':'total_area'})
            viewdata = viewdata[:5]
            viewlst = []
            for viewdd in viewdata:
                urllst = viewdd.find_all('a')
                for url in urllst:
                    if 'api_txt_lines' in url['class']:
                        if 'adcr.naver.com' in url['href']:
                            content = '광고'
                        elif 'blog.naver.com' in url['href']:
                            content = '블로그'
                        elif 'post.naver.com' in url['href']:
                            content = '포스트'
                        else:
                            content = '카페'
                        viewlst.append(content)
        except:
            dic['view(인플루언서)'] = '없음'
    
    for tab in ['네이버쇼핑', '지식iN']:
        try:
            dic[tab] = pagelst2.index(tab) + 1
        except:
            dic[tab] = '없음'
    if dic['인플루언서 노출 여부'] == 'x':
        for iidd in range(len(viewlst)):
            dic['view ' + str(iidd + 1) + ' 순위'] = viewlst[iidd]
    
    resultlst.append(dic)
    print(dic)
    print(kw)
#     print()

resultdf = pd.DataFrame(resultlst)
resultdf

잉
알림
잉
코로나19
{'키워드': '혈압정상수치', '인플루언서 노출 여부': 'o', 'view(인플루언서)': 1, '네이버쇼핑': '없음', '지식iN': 3}
혈압정상수치
잉
인기 주제 둘러보기
고지혈증 원인
인플루언서 참여 콘텐츠
고지혈증 수치
'고지혈증' 관련 브랜드 콘텐츠
잉
고지혈증 FAQ
코로나19
{'키워드': '고지혈증', '인플루언서 노출 여부': 'o', 'view(인플루언서)': 5, '네이버쇼핑': '없음', '지식iN': '없음'}
고지혈증
지식스니펫이 정보가 표시된 이유
잉
코로나19
{'키워드': '혈압낮추는법', '인플루언서 노출 여부': 'x', 'view(인플루언서)': 3, '네이버쇼핑': '없음', '지식iN': 5, 'view 1 순위': '광고', 'view 2 순위': '광고', 'view 3 순위': '카페', 'view 4 순위': '블로그', 'view 5 순위': '블로그'}
혈압낮추는법
잉
인기 주제 둘러보기
고혈압에 좋은 음식
고혈압 진단
고혈압 증상
'고혈압' 관련 브랜드 콘텐츠
잉
고혈압 FAQ
코로나19
{'키워드': '고혈압', '인플루언서 노출 여부': 'x', 'view(인플루언서)': '없음', '네이버쇼핑': '없음', '지식iN': '없음', 'view 1 순위': '광고', 'view 2 순위': '광고', 'view 3 순위': '카페', 'view 4 순위': '블로그', 'view 5 순위': '블로그'}
고혈압
지식스니펫이 정보가 표시된 이유
알림
잉
코로나19
{'키워드': '고혈압증상', '인플루언서 노출 여부': 'o', 'view(인플루언서)': 3, '네이버쇼핑': '없음', '지식iN': 5}
고혈압증상
알림
잉
코로나19
{'키워드': '혈당정상수치', '인플루언서 노출 여부': 'o', 'view(인플루언서)': 2, '네이버쇼핑': '없음', '지식iN': 6}
혈당정상수치
지식스니펫이 정보가 표시된 이유
알림
잉
코로나19
{'키워드': '고혈압낮추는방법',

,키워드,인플루언서 노출 여부,view(인플루언서),네이버쇼핑,지식iN,view 1 순위,view 2 순위,view 3 순위,view 4 순위,view 5 순위
0,혈압정상수치,o,1,없음,3,NaN,NaN,NaN,NaN,NaN
1,고지혈증,o,5,없음,없음,NaN,NaN,NaN,NaN,NaN
2,혈압낮추는법,x,3,없음,5,광고,광고,카페,블로그,블로그
3,고혈압,x,없음,없음,없음,광고,광고,카페,블로그,블로그
4,고혈압증상,o,3,없음,5,NaN,NaN,NaN,NaN,NaN
5,혈당정상수치,o,2,없음,6,NaN,NaN,NaN,NaN,NaN
6,고혈압낮추는방법,o,3,없음,6,NaN,NaN,NaN,NaN,NaN
7,고지혈증좋은음식,o,3,없음,6,NaN,NaN,NaN,NaN,NaN
8,혈압,o,6,없음,없음,NaN,NaN,NaN,NaN,NaN
9,고혈압에좋은음식,o,3,없음,6,NaN,NaN,NaN,NaN,NaN


In [15]:
resultdf.to_excel("키워드 전략_220526.xlsx")
resultdf

,키워드,인플루언서 노출 여부,view(인플루언서),네이버쇼핑,지식iN,view 1 순위,view 2 순위,view 3 순위,view 4 순위,view 5 순위
0,혈압정상수치,o,1,없음,3,NaN,NaN,NaN,NaN,NaN
1,고지혈증,o,5,없음,없음,NaN,NaN,NaN,NaN,NaN
2,혈압낮추는법,x,3,없음,5,광고,광고,카페,블로그,블로그
3,고혈압,x,없음,없음,없음,광고,광고,카페,블로그,블로그
4,고혈압증상,o,3,없음,5,NaN,NaN,NaN,NaN,NaN
5,혈당정상수치,o,2,없음,6,NaN,NaN,NaN,NaN,NaN
6,고혈압낮추는방법,o,3,없음,6,NaN,NaN,NaN,NaN,NaN
7,고지혈증좋은음식,o,3,없음,6,NaN,NaN,NaN,NaN,NaN
8,혈압,o,6,없음,없음,NaN,NaN,NaN,NaN,NaN
9,고혈압에좋은음식,o,3,없음,6,NaN,NaN,NaN,NaN,NaN


In [16]:
from datetime import timedelta, datetime
import time
import random
import requests
import pandas as pd
import hashlib
import hmac
import base64


class Signature:

    @staticmethod
    def generate(timestamp, method, uri, secret_key):
        message = "{}.{}.{}".format(timestamp, method, uri)
        hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)

        hash.hexdigest()
        return base64.b64encode(hash.digest())

def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 
            'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 
            'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

BASE_URL = 'https://api.naver.com'
API_KEY = '0100000000af2ea938d5dc983af8448c555ee9a0aacbacd9cc0a2a4e213ff85d6a45db797e'
SECRET_KEY = 'AQAAAACvLqk41dyYOvhEjFVe6aCqXh4WuJhTkTSy2FglsvtI3A=='
CUSTOMER_ID = '1519222'

totaldf = [{'keyword': '','data':'', 'searchCount':''}]
totaldf = pd.DataFrame(totaldf)


for keyword in keywordlst:
# for keyword in ['치매예방뇌영양제', '60대영양제추천', '건망증영양제', '집중력향상영양제', '천마영양제', '집중약']:
    keyword = keyword.replace(' ','')
    uri = '/keywordstool'
    method = 'GET'
    r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
                     headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

    print(keyword)
    status = r.json()['keywordList'][0]
    if status['monthlyPcQcCnt'] == '< 10':
        status['monthlyPcQcCnt'] = 0
    if status['monthlyMobileQcCnt'] == '< 10':
        status['monthlyMobileQcCnt'] = 0
        
    if status['monthlyPcQcCnt'] == 0 and status['monthlyMobileQcCnt'] == 0:
        pass
    else:
        totalCnt = status['monthlyPcQcCnt'] + status['monthlyMobileQcCnt']

        import os
        import sys
        import urllib.request
        client_id = "LYZVDwt9_Sdi5523ujrA"
        client_secret = "FLi5QKPYgv"
        url = "https://openapi.naver.com/v1/datalab/search";
        now = datetime.now()
        startDate = (now-timedelta(days=30)).strftime('20%y-%m-%d')
        endDate = (now-timedelta(days=1)).strftime('20%y-%m-%d')
        oneweekago = (now-timedelta(days=7)).strftime('20%y-%m-%d')
        onedayago = (now-timedelta(days=1)).strftime('20%y-%m-%d')

        daylist = []
        for i in range(1,8):
            days = (now-timedelta(days=i)).strftime('20%y-%m-%d')
            daylist.append(days)

        body = '{\"startDate\":\"'+'{}'.format(startDate)+'\",\"endDate\":\"'+'{}'.format(endDate)+'\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"'+'{}'.format(keyword)+'\",\"keywords\":[\"'+'{}'.format(keyword)+'\"]}]}';

        # ,\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"
        # 궁금한건 https://developers.naver.com/docs/serviceapi/datalab/search/search.md#python
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            result = response_body.decode('utf-8')

        else:
            print("Error Code:" + rescode)


        import ast
        # import json
        # my_json = response_body.decode('utf8').replace("'", '"')
        # data = json.loads(my_json)
        # print(data['startDate'])
        import pandas as pd
        a = ast.literal_eval(result)
        searchdata = pd.DataFrame(a['results'][0]['data'])
        searchdata['keyword'] = keyword


        import numpy as np
        try:
            ratiosum = int(np.sum(searchdata['ratio']))
            totalratio = totalCnt/ratiosum


            searchdata['searchCount'] = searchdata['ratio'].apply(lambda x : round(x*totalratio))
            searchdata = searchdata[['keyword','period', 'searchCount']]
            searchdata.columns = ['keyword','data', 'searchCount']

            searchdata =searchdata.loc[searchdata['data'].isin(daylist)]
            totaldf = pd.concat([totaldf, searchdata])
            time.sleep(5)
        except KeyError:
            pass
totaldf = totaldf[1:]
totaldf

혈압정상수치
고지혈증
혈압낮추는법
고혈압
고혈압증상
혈당정상수치
고혈압낮추는방법
고지혈증좋은음식
혈압
고혈압에좋은음식
고지혈증증상
고지혈증약
혈압높을때증상
혈압약
고혈압원인
고혈압약
혈압낮추는음식
심혈관질환
혈당낮추는음식
고지혈증약부작용
혈당낮추는법
혈압에좋은음식
심혈관질환증상
고지혈증원인
뇌출혈증상
고혈압식단
혈압에좋은영양제
고지혈증에좋은음식
고혈압에좋은영양제
고혈압관리
혈압영양제
뇌경색
뇌경색전조증상
뇌경색초기증상
뇌경색치료
뇌경색증상
뇌경색후유증
뇌경색원인
뇌경색좋은음식
뇌출혈초기증상
뇌출혈
뇌출혈전조증상
뇌출혈원인
뇌출혈증상
뇌출혈후유증
뇌졸중초기증상
뇌졸중
뇌졸중전조증상
뇌졸증
뇌졸증증상
뇌졸중증상
뇌졸증전조증상
뇌졸중원인
뇌졸증증세


,keyword,data,searchCount
23,혈압정상수치,2022-05-19,3431
24,혈압정상수치,2022-05-20,3356
25,혈압정상수치,2022-05-21,2833
26,혈압정상수치,2022-05-22,1589
27,혈압정상수치,2022-05-23,3549
...,...,...,...
25,뇌졸증증세,2022-05-21,22
26,뇌졸증증세,2022-05-22,23
27,뇌졸증증세,2022-05-23,13
28,뇌졸증증세,2022-05-24,32


In [17]:
stadf = totaldf.copy()

In [18]:
resultdf['키워드 주간 검색량'] = resultdf['키워드'].apply(lambda x: sum(stadf.loc[stadf['keyword'] == x.replace(' ','')]['searchCount']))
resultdf

,키워드,인플루언서 노출 여부,view(인플루언서),네이버쇼핑,지식iN,view 1 순위,view 2 순위,view 3 순위,view 4 순위,view 5 순위,키워드 주간 검색량
0,혈압정상수치,o,1,없음,3,NaN,NaN,NaN,NaN,NaN,21628
1,고지혈증,o,5,없음,없음,NaN,NaN,NaN,NaN,NaN,19932
2,혈압낮추는법,x,3,없음,5,광고,광고,카페,블로그,블로그,12698
3,고혈압,x,없음,없음,없음,광고,광고,카페,블로그,블로그,11625
4,고혈압증상,o,3,없음,5,NaN,NaN,NaN,NaN,NaN,11023
5,혈당정상수치,o,2,없음,6,NaN,NaN,NaN,NaN,NaN,11983
6,고혈압낮추는방법,o,3,없음,6,NaN,NaN,NaN,NaN,NaN,10250
7,고지혈증좋은음식,o,3,없음,6,NaN,NaN,NaN,NaN,NaN,8774
8,혈압,o,6,없음,없음,NaN,NaN,NaN,NaN,NaN,8510
9,고혈압에좋은음식,o,3,없음,6,NaN,NaN,NaN,NaN,NaN,6658


In [19]:
resultdf.to_excel("키워드 전략 분석 데이터_220526.xlsx", index=False)